# TensorFlow Training at Scale

This notebook is tested using `Data Science - Python 3 Kernel` running on a `ml.t3.medium` instance. Please ensure that you use `Python 3 (Data Science)` in the top right on your notebook.

## Loading stored variables
If you ran this notebook before, you may want to re-use the resources you aready created with AWS. Run the cell below to load any prevously created variables. You should see a print-out of the existing variables. If you don't see anything printed then it's probably the first time you are running the notebook!

In [ ]:
%store -r
%store

In [ ]:
# Ensure updated SageMaker SDK version
%pip install -U -q sagemaker

In [ ]:
# Install SageMaker Experiments Library
# https://github.com/aws/sagemaker-experiments
%pip install sagemaker-experiments

In [ ]:
%%writefile tf_train.py

import os
import argparse

import tensorflow as tf
from tensorflow.keras.experimental import LinearModel, WideDeepModel
from tensorflow import keras


def parse_args():

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--learning_rate", type=float, default=0.1)

    # data directories
    parser.add_argument("--training", type=str, default=os.environ["SM_CHANNEL_TRAINING"])
    parser.add_argument("--testing", type=str, default=os.environ["SM_CHANNEL_TESTING"])

    # model directory: we will use the default set by SageMaker, /opt/ml/model
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))

    return parser.parse_known_args()


def get_train_data(train_dir, batch_size):

    def pack(features, label):
        linear_features = [tf.cast(features['day_of_week'], tf.float32), tf.cast(features['month'], tf.float32),
                           tf.cast(features['hour'], tf.float32), features["trip_distance"]]
        
        dnn_features = [features['pickup_latitude'], features['pickup_longitude'], features['dropoff_latitude'], features['dropoff_longitude'], features["trip_distance"]]
        return (tf.stack(linear_features, axis=-1), tf.stack(dnn_features, axis=-1)), label

    
    column_headers = ["day_of_week","month","hour","pickup_latitude","pickup_longitude",
                      "dropoff_latitude","dropoff_longitude","trip_distance","fare_amount"]

    ds = tf.data.experimental.make_csv_dataset(tf.io.gfile.glob(train_dir + '/*.csv'),
                                               batch_size=batch_size,
                                               column_names=column_headers,
                                               num_epochs=1,
                                               shuffle=True,
                                               label_name="fare_amount")
    ds = ds.map(pack)
    return ds



if __name__ == "__main__":
    args, _ = parse_args()
    
    print(args)
    print(os.environ)


    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate
    train_dir = args.training
    
    ds = get_train_data(train_dir, batch_size)
    
    linear_model = LinearModel()
    dnn_model = keras.Sequential(
        [keras.layers.Dense(units=64), keras.layers.Dense(units=1)]
    )
    combined_model = WideDeepModel(linear_model, dnn_model)
    combined_model.compile(optimizer="Adam", loss="mse", metrics=["mse"])
    combined_model.fit(ds, epochs=epochs)

In [ ]:
import time
import boto3

sess = boto3.Session()
sm = sess.client("sagemaker")

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

taxi_experiment = Experiment.create(
    experiment_name=f"tf2-taxi-wide-deep-{int(time.time())}",
    description="Training wide and deep model on nyc taxi dataset",
    sagemaker_boto_client=sm,
)
print(taxi_experiment)

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()

output_bucket = f"s3://{bucket}/nyc-taxi/model/"


# Create unique job name with hyperparameter and time
run_number = 1
time_append = int(time.time())
training_job_name = f"tf2-taxi-wide-deep-training-{time_append}"
trial_name = f"trial-tf2-taxi-wide-deep-training-{time_append}"
trial_desc = f"tf2-taxi-wide-deep-run-{run_number}"
    

# Create a new Trial and associate Tracker to it
tf2_taxi_trial = Trial.create(
    trial_name=trial_name,
    experiment_name=taxi_experiment.experiment_name,
    sagemaker_boto_client=sm,
    tags=[{"Key": "trial-desc", "Value": trial_desc}],
)


# Create an experiment config that associates training job to the Trial
experiment_config = {
    "ExperimentName": taxi_experiment.experiment_name,
    "TrialName": tf2_taxi_trial.trial_name,
    "TrialComponentDisplayName": training_job_name,
}


metric_definitions = [
    {"Name": "mse", "Regex": "mse: ([0-9\\.]+)"},
    {"Name": "val_mse", "Regex": "val_mse: ([0-9\\.]+)"},
]


spot_training=True

tf_estimator = TensorFlow(
    entry_point="tf_train.py",
    base_job_name="tf2-taxi-wide-deep",
    role=role,
    use_spot_instances=spot_training,
    max_run=1800,  # max training time in seconds
    max_wait=1800,  # seconds to wait for spot instance
    framework_version="2.6.2",
    py_version="py38",
    metric_definitions=metric_definitions,
    input_mode="File",
    output_path=output_bucket,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    hyperparameters={"batch_size": 512, "epochs": 1}
)

In [ ]:
tf_estimator.fit({"training": f"s3://{data_bucket}/train/", "testing": f"s3://{data_bucket}/test/"}, logs=False) 